In [ ]:
from enum import Enum
from ..flux import can

IS = Enum("IS", "working, destroyed, done, ready, broken") 
BE = Enum("BE", "selected, destroyed, used, repaired, cancelled")
HAS = Enum("HAS", "amount")
OF = Enum("OF", "water, meat, paprica")

@can(BE.destroyed)
def destroy(self, thing):
    thing.flux_to(IS.destroyed)
    print("%s broke the %s!!!" % (self.__class__.__name__, thing.name))

In [ ]:
from enum import Enum

class IS(Enum):
	starting = 1
	connecting = 2
	ready = 3
	disconnected = 4
	down = 5

class BE(Enum):
	prepared = 1
	etc = 2

In [ ]:
from enum import Enum

IS = Enum("STATE", "loading login signup playing exiting")
CAN = Enum("CAN", "start navigate play configure")
BE  = Enum("BE", "closed clicked")
HAS = Enum("HAS", "clicks")

In [ ]:
from enum import Enum

IS = Enum('IS', 'closed, open, destroyed')
BE = Enum("BE", "opened, closed, written, entered, fubared")
HAS = Enum("HAS", "been_opened, hitpoints, been_fluxed")